# 네이버 영화 리뷰 감정분석 실습

- 데이터 출처 : <a href='https://github.com/e9t/nsmc'>Github<a>

In [41]:
import pandas as pd
import numpy as np
import os
os.chdir('/Users/younghun/Desktop/gitrepo/data')

In [42]:
train_df = pd.read_csv('korean_train.txt', sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [43]:
# label종류 보기
train_df['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [44]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [45]:
import re

# document에 결측치가 존재하긴 하지만 5개 밖에 존재하지 않음
# 결측치를 공백으로 채워주기
train_df = train_df.fillna(' ')

# 정규 표현식 이용해서 리뷰의 모든 숫자 텍스트 공백으로 바꾸어주기
train_df['document'] = train_df['document'].apply(lambda x : re.sub('\d+', ' ',x))
train_df = train_df.drop('id', axis=1)

# Test 데이터에 대해서도 적용하기
test_df = pd.read_csv('korean_test.txt', sep='\t')
test_df = test_df.fillna(' ')
test_df['documnet'] = test_df['document'].apply(lambda x : re.sub('\d+',' ',x))
test_df = test_df.drop('id', axis=1)

In [46]:
# konlpy를 이용해서 토큰화시키기
# 데이터가 댓글 종류의 텍스트라서 Twitter 품사 태깅(POS) 클래스를 활용해보자.
from konlpy.tag import Twitter

twitter = Twitter()
# 추후에 텍스트를 벡터화시키는 과정에서 토큰화함수로 사용하기 위해 함수로 정의
def tw_tokenize(text):
    # Twitter()는 입력인자로 들어온 텍스트를 어근단위로 토큰화시켜 리스트로 반환!
    # morphs함수를 사용!
    tokens_ko = twitter.morphs(text)
    return tokens_ko

# example test
tw_tokenize('아버지가 방에 들어가신다')

/Users/younghun/opt/anaconda3/envs/venvforpython/lib/python3.7/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['아버지', '가', '방', '에', '들어가신다']

In [47]:
# 텍스트를 토큰화시키면서 feature 벡터화 시키기
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# 위 셀에서 정의해준 토큰화 함수를 벡터화시키는 함수에 인자로 넣어주기
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenize,
                            ngram_range=(1,2), min_df=3,
                            max_df=0.9)
train_tfidf = tfidf_vect.fit_transform(train_df['document'])

In [48]:
# Logistic Regression이용해서 감성분석 분류 수행
lg_clf = LogisticRegression(random_state=42)

# GridSearchCV하기 위해 튜닝할 여러가지 파라미터 정의
# C값(1/alpha)이 작을수록 Regularization 강도가 큼!
params = {'C':[1, 3, 5, 10, 20]}
grid_cv = GridSearchCV(lg_clf, param_grid=params, cv=3,
                      scoring='accuracy', verbose=1)
# 텍스트를 벡터화시킨 데이터들로 학습
grid_cv.fit(train_tfidf, train_df['label'])
print("최적의 파라미터: ", grid_cv.best_params_)
print("최고 성능: ", round(grid_cv.best_score_, 4))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/younghun/opt/anaconda3/envs/venvforpython/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/younghun/opt/anaconda3/envs/venvforpython/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.h

최적의 파라미터:  {'C': 3}
최고 성능:  0.8593


/Users/younghun/opt/anaconda3/envs/venvforpython/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [49]:
# Test 데이터로 모델 최종 성능 평가
from sklearn.metrics import accuracy_score

# Test 데이터 feature 벡터화시키기!
# 이 때, Train 데이터에 적용했던 벡터화도구를 그대로 쓰기 위해 transform만 해주어야 함!
test_tfidf = tfidf_vect.transform(test_df['document'])

best_estimator = grid_cv.best_estimator_
test_preds = best_estimator.predict(test_tfidf)
acc = accuracy_score(test_df['label'], test_preds)
print(f"Logistic Regression 정확도: {acc : .4f}")

Logistic Regression 정확도:  0.8619


- 원본 데이터 로드할 때 원본 데이터의 오탈자가 없는지도 확인해보자. 
- 해당 error message : ``classification metrics can't handle a mix of unknown and binary targets``